In [58]:
from lxml import etree

# Read the XML file as bytes
with open("экспорт.xml", "rb") as file:
    xml_content = file.read()

# Parse the XML content
root = etree.fromstring(xml_content)

# Define namespace mappings
namespaces = {
    'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main',
    'v': 'urn:schemas-microsoft-com:vml'
}

# Extract and print text from textboxes
for textbox in root.xpath('.//v:textbox', namespaces=namespaces):
    for text_elem in textbox.xpath('.//w:t', namespaces=namespaces):
        print(text_elem.text)


Квитанция взвешивания от 28.08.23 15:09
ООО
 "ТД"Карелия
 Неруд"
Дата|Время
№
 вагона
--------Вид
 груза--------
Груз-сть
Брутто
Тара
Нетто
Пр|Нд
Скор.
Поставщик|Адресат
  №
  
Тел.
 1
Тел.
 2
22.08.23
 12:08
Тепловоз
 ТЭМ2
115,80
-4,5
22.08.23
 12:08
1,1
57,90
58,00
22.08.23
 12:08
69,50
80,40
23,30
57,10
12,40
-4,6
22.08.23
 12:08
1,2
40,80
39,60
22.08.23
 12:09
87,40
-4,7
22.08.23
 12:09
1,3
42,70
44,70
22.08.23
 12:09
85,70
-4,7
22.08.23
 12:09
1,4
43,80
41,90
22.08.23
 12:09
87,90
-4,7
22.08.23
 12:09
1,5
44,60
43,30
22.08.23
 12:09
83,90
-4,9
22.08.23
 12:09
1,6
41,60
42,40
22.08.23
 12:09
86,40
-5,1
22.08.23
 12:09
1,7
43,20
43,20
22.08.23
 12:09
86,90
-5,2
22.08.23
 12:09
1,8
42,50
44,40
22.08.23
 12:10
86,50
-5,3
22.08.23
 12:10
1,9
42,80
43,60
22.08.23
 12:10
85,90
-5,4
22.08.23
 12:10
1,10
42,40
43,50
22.08.23
 12:10
86,20
-5,4
22.08.23
 12:10
1,11
44,30
41,90
22.08.23
 12:10
87,00
-5,5
22.08.23
 12:10
1,12
44,10
42,90
22.08.23
 12:10
86,30
-5,5
22.08.23
 12:10
1,13
42,10
44

In [130]:
import re

# Read the contents of the file

def check_date_format(s):
    pattern = r'^\d{2}\.\d{2}\.\d{2}$'
    return bool(re.match(pattern, s))

def find_all_date_y(results):
    y_positions = []
    for item in results:
        text = item['text']
        x = item['position']['margin-left']
        y = item['position']['margin-top']
        if check_date_format(text) and x > 0 and y > 0 and item['page'] == 1:
            y_positions.append(y)
            print(text, y)

    y_positions = sorted(list(set(y_positions)))
    return y_positions

# print(find_all_date_y())

In [128]:
import xml.etree.ElementTree as ET

# Parse the XML
tree = ET.parse('экспорт.xml')
root = tree.getroot()

# Define the XML namespaces used in your XML
namespaces = {
    'v': 'urn:schemas-microsoft-com:vml',
    'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'
}

WRITE = False
# min_y = min(find_all_date_y(results))
# print(min_y)
# y_offset = min_y
page = 0
# Iterate over the text boxes in the XML and extract position and text
results = []
for rect in root.findall(".//v:rect", namespaces):
    style = rect.get('style')
    # Extract position data from style attribute
    position_data = {}
    for pair in style.split(';'):
        name, _, value = pair.partition(':')
        position_data[name] = value

    # Get text data
    text_content = rect.find(".//w:t", namespaces)
    if text_content is not None:
        text = text_content.text

        if 'Страница' in text:
            WRITE = False
            # reset this result y values to start from 0 by finding the smallest y and subtracting it from every y value
            # add y_offset ()
            # min_y = min(find_all_date_y(results))
            # print(min_y)
            # y_offset = min_y

        if WRITE:
            ml = pt_in_2_value(position_data['margin-left'])
            mt = pt_in_2_value(position_data['margin-top'])
            w = pt_in_2_value(position_data['width'])
            h = pt_in_2_value(position_data['height'])

            if not (ml >= 0 and mt >= 0 and w >= 0 and h >= 0):
                continue

            position = {'margin-left': ml,
                        'margin-top': mt,
                        'width': w,
                        'height': h}

            results.append({
                'text': text,
                'position': position,
                'page': page
            })

        if text == ' 2':
            WRITE = True
            page += 1

# Print the results
for item in results:
    print(item['text'])
    print("Position:", item['position']['margin-left'],
          item['position']['margin-top'], item['page'])

Тепловоз
Position: 163 82 1
 ТЭМ2
Position: 207 82 1
115,80
Position: 331 82 1
-4,5
Position: 485 83 1
22.08.23
Position: 42 82 1
 12:08
Position: 81 82 1
1,1
Position: 16 82 1
57,90
Position: 517 83 1
58,00
Position: 553 83 1
69,50
Position: 298 97 1
80,40
Position: 337 97 1
23,30
Position: 374 97 1
57,10
Position: 408 97 1
12,40
Position: 443 97 1
-4,6
Position: 485 98 1
22.08.23
Position: 42 97 1
 12:08
Position: 81 97 1
1,2
Position: 16 97 1
40,80
Position: 517 98 1
39,60
Position: 553 98 1
87,40
Position: 337 111 1
-4,7
Position: 485 112 1
22.08.23
Position: 42 111 1
 12:09
Position: 81 111 1
1,3
Position: 16 111 1
42,70
Position: 517 112 1
44,70
Position: 553 112 1
85,70
Position: 337 125 1
-4,7
Position: 485 127 1
22.08.23
Position: 42 125 1
 12:09
Position: 81 125 1
1,4
Position: 16 125 1
43,80
Position: 517 127 1
41,90
Position: 553 126 1
87,90
Position: 337 140 1
-4,7
Position: 485 141 1
22.08.23
Position: 42 140 1
 12:09
Position: 81 140 1
1,5
Position: 16 140 1
44,60
Positi

In [119]:
import re

# Read the contents of the file
with open("clear_position.txt", "r") as file:
    data = file.read()

# Find all the y-positions corresponding to dates in the specified format
date_pattern = r'(\d{2}\.\d{2}\.\d{2})\s+Position: \d+pt (\d+\.?\d*)pt'
matches = re.findall(date_pattern, data)

# Grouping y-positions by date
date_to_y_positions = []
for date, y in matches:
    date_to_y_positions.append(float(y))
date_to_y_positions = sorted(list(set(date_to_y_positions)))
print(date_to_y_positions)


In [131]:
import matplotlib.pyplot as plt
import re

# Split the x and y coordinates
x_coords = [float(x) for x, _ in positions]
y_coords = [float(y) for _, y in positions]

# Normalize if necessary (depending on the size of your canvas and the range of your coordinates)
# Here, as an example, I'm multiplying by a factor for visualization. Adjust as needed.
x_coords = [x for x in x_coords]
y_coords = [y for y in y_coords]

plt.figure(figsize=(10, 10))
# plt.scatter(x_coords, y_coords)
# Plot points with color based on 'page'
for item in results:
    x = item['position']['margin-left']
    y = item['position']['margin-top']
    if item['page'] == 1:
        plt.scatter(x, y, color='blue')
    elif item['page'] == 2:
        plt.scatter(x, y, color='green')

date_to_y_positions = find_all_date_y(results)

x_borders = [x - 4 for x in [0, 42, 81, 163, 298, 331, 374, 408, 443, 485, 517, 553]]
y_borders = [y - 7 for y in date_to_y_positions]

# Draw several vertical and horizontal lines
for x_line in x_borders:  # You can change these values to your needs
    plt.axvline(x=x_line, color='red', linestyle='--', linewidth=0.8)

for y_line in y_borders:  # Again, adjust these values as needed
    plt.axhline(y=y_line, color='red', linestyle='--', linewidth=0.8)


# Set x-axis limits
# plt.xlim(0, 25)

plt.gca().invert_yaxis()  # This will make the top left corner (0,0) as is common with many graphics systems
plt.title('Positions on Canvas')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.grid(True)
plt.show()


KeyError: 'x'

<Figure size 1000x1000 with 0 Axes>

In [14]:
import pandas as pd

# List of desired column names
columns = ["№", "Дата", "Время", "№ вагона", "Вид груза", "Груз-сть", "Брутто", "Тара", "Нетто", "Пр|Нд", "Скор.", "Тел. 1", "Тел. 2"]  # Add more names if required

# Create an empty DataFrame with the desired column names
df = pd.DataFrame(columns=columns)

print(df)

# Print the results
# for item in results:
#     print(item['text'])
#     print("Position:", item['position']['margin-left'], item['position']['margin-top'])

for y in y_borders:
    for x in x_borders:
        for item in results:
            ml = item['position']['margin-left'] 
            mx = item['position']['margin-top']
            # if ml > x and mx > y:
            #     print(x,y,ml,mx)
            print(ml, mx)
        

Empty DataFrame
Columns: [№, Дата, Время, № вагона, Вид груза, Груз-сть, Брутто, Тара, Нетто, Пр|Нд, Скор., Тел. 1, Тел. 2]
Index: []
257pt 37pt
9in 18pt
676pt 18pt
751pt 18pt
42pt 66pt
108pt 66pt
120pt 66pt
164pt 66pt
3in 66pt
280pt 66pt
328pt 66pt
368pt 66pt
400pt 66pt
437pt 66pt
480pt 67pt
620pt 66pt
16pt 66pt
35pt 66pt
515pt 67pt
538pt 67pt
551pt 67pt
574pt 67pt
-1in 37pt
-33pt 37pt
163pt 82pt
207pt 82pt
331pt 82pt
485pt 83pt
42pt 82pt
81pt 82pt
16pt 82pt
517pt 83pt
553pt 83pt
-1in 52pt
-33pt 52pt
298pt 97pt
337pt 97pt
374pt 97pt
408pt 97pt
443pt 97pt
485pt 98pt
42pt 97pt
81pt 97pt
16pt 97pt
517pt 98pt
553pt 98pt
-1in 66pt
-33pt 66pt
337pt 111pt
485pt 112pt
42pt 111pt
81pt 111pt
16pt 111pt
517pt 112pt
553pt 112pt
-1in 80pt
-33pt 80pt
337pt 125pt
485pt 127pt
42pt 125pt
81pt 125pt
16pt 125pt
517pt 127pt
553pt 126pt
-1in 95pt
-33pt 95pt
337pt 140pt
485pt 141pt
42pt 140pt
81pt 140pt
16pt 140pt
517pt 141pt
553pt 141pt
-1in 109pt
-33pt 109pt
337pt 154pt
485pt 155pt
42pt 154pt
81pt 154pt
